In [1]:
from dotenv import load_dotenv
load_dotenv()

from argparse import ArgumentParser
import warnings
from collections import OrderedDict
import json
import io
import os
import sys
import pickle
import base64
from traceback import print_exc

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data.sampler as sampler
from torchvision.transforms import Compose, Normalize, ToTensor

from sklearn.cluster import KMeans

from tqdm import tqdm
from time import sleep, time

import logging

from non_iid_generator.customDataset import CustomDataset

DEVICE = os.environ["TORCH_DEVICE"]
DEVICE = "cuda"


In [2]:
def load_data(train_dataset_path, test_dataset_path):
    """Load CIFAR-10 (training and test set)."""

    batch_size = 128
    momentum = 0.9
    weight_decay = 1e-4
    finetune_lr = 0.001

    train_data = pickle.load(open(train_dataset_path, "rb"))
    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True)

    test_data = pickle.load(open(test_dataset_path, "rb"))
    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True)

    return train_loader, test_loader

def fine_tune(model, iterations, train_loader, print_frequency=100):
    '''
        short-term fine-tune a simplified model
        
        Input:
            `model`: model to be fine-tuned.
            `iterations`: (int) num of short-term fine-tune iterations.
            `print_frequency`: (int) how often to print fine-tune info.
        
        Output:
            `model`: fine-tuned model.
    '''

    # Data loaders for fine tuning and evaluation.
    batch_size = 128
    momentum = 0.9
    weight_decay = 1e-4
    finetune_lr = 0.001

    criterion = torch.nn.BCEWithLogitsLoss()
    
    _NUM_CLASSES = 10
    optimizer = torch.optim.SGD(
        model.parameters(),
        finetune_lr, 
        momentum=momentum,
        weight_decay=weight_decay)

    model = model.to(DEVICE)
    model.train()
    dataloader_iter = iter(train_loader)
    for i in range(iterations):
        try:
            (input, target) = next(dataloader_iter)
        except:
            dataloader_iter = iter(train_loader)
            (input, target) = next(dataloader_iter)
            
        if i % print_frequency == 0:
            print('Fine-tuning iteration {}'.format(i))
            sys.stdout.flush()
        
        # Ensure the target shape is sth like torch.Size([batch_size])
        if len(target.shape) > 1: target = target.reshape(len(target))

        target.unsqueeze_(1)
        target_onehot = torch.FloatTensor(target.shape[0], _NUM_CLASSES)
        target_onehot.zero_()
        target_onehot.scatter_(1, target, 1)
        target.squeeze_(1)
        input, target = input.to(DEVICE), target.to(DEVICE)
        target_onehot = target_onehot.to(DEVICE)

        pred = model(input)
        loss = criterion(pred, target_onehot)
        optimizer.zero_grad()
        loss.backward()  # compute gradient and do SGD step
        optimizer.step()

    return model

In [3]:
model = torch.load("models/alexnet/model_cpu.pth.tar")

In [4]:
weights_list = []
for client_id in range(10):
    print(f">> Dataset No: {client_id}")
    train_dataset_path = f"./data/Cifar10/train/{client_id}.pkl"
    test_dataset_path = f"./data/Cifar10/test/{client_id}.pkl"
    trainloader, testloader = load_data(train_dataset_path, test_dataset_path)

    temp_model = fine_tune(model, 10, trainloader, print_frequency=1)
    weights_model = np.concatenate([param.data.cpu().numpy().flatten() for param in model.parameters()])

    weights_list.append(weights_model)

>> Dataset No: 0
Fine-tuning iteration 0
Fine-tuning iteration 1
Fine-tuning iteration 2
Fine-tuning iteration 3
Fine-tuning iteration 4
Fine-tuning iteration 5
Fine-tuning iteration 6
Fine-tuning iteration 7
Fine-tuning iteration 8
Fine-tuning iteration 9
>> Dataset No: 1
Fine-tuning iteration 0
Fine-tuning iteration 1
Fine-tuning iteration 2
Fine-tuning iteration 3
Fine-tuning iteration 4
Fine-tuning iteration 5
Fine-tuning iteration 6
Fine-tuning iteration 7
Fine-tuning iteration 8
Fine-tuning iteration 9
>> Dataset No: 2
Fine-tuning iteration 0
Fine-tuning iteration 1
Fine-tuning iteration 2
Fine-tuning iteration 3
Fine-tuning iteration 4
Fine-tuning iteration 5
Fine-tuning iteration 6
Fine-tuning iteration 7
Fine-tuning iteration 8
Fine-tuning iteration 9
>> Dataset No: 3
Fine-tuning iteration 0
Fine-tuning iteration 1
Fine-tuning iteration 2
Fine-tuning iteration 3
Fine-tuning iteration 4
Fine-tuning iteration 5
Fine-tuning iteration 6
Fine-tuning iteration 7
Fine-tuning iteratio

In [23]:
all_weights = np.vstack(weights_list)
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_assignments = kmeans.fit_predict(all_weights)

In [6]:
all_weights = np.vstack(weights_list)


In [8]:
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_assignments = kmeans.fit_predict(all_weights)

KeyboardInterrupt: 